# Modeling ODEs Via Simulation

In [ ]:
from math import *
from time import time
from datetime import timedelta
from random import random
import numpy as np
#Sympy
from sympy import *
from sympy.plotting import plot3d
%matplotlib inline
from IPython.display import display
init_printing()
#visualization
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Legend
output_notebook()

In [ ]:
cro, cI, a_cro, a_cI, c_cro, c_cI, gamma, t = symbols("[cro] [cI] alpha_cro alpha_cI c_cro c_cI gamma t", positive=1)
dot_cro = a_cro/(1+(cI/c_cI)**2)**2 - gamma*cro
dot_cI = a_cI/(1+(cro/c_cro)**2) - gamma*cI
display(Eq(Symbol("\dot{[cro]}"), dot_cro), Eq(Symbol("\dot{[cI]}"), dot_cI))

In [ ]:
alpha_cro=.2
alpha_cI=.15
rep_cro=8
rep_cI=2
gamma_cro = .001
gamma_cI = .004
subs_cro = dot_cro.subs([(a_cro, alpha_cro), (c_cI, rep_cI), (gamma, gamma_cro)])
subs_cI = dot_cI.subs([(a_cI, alpha_cI), (c_cro, rep_cro), (gamma, gamma_cI)])
plot3d(subs_cro, (cro, 0, 40), (cI, 0, 40))
plot3d(subs_cI, (cro, 0, 40), (cI, 0, 40))

In [ ]:
class Cell:
    def __init__(self, plot_name=None, stochastic=0, continuous_plotting=1):
        self.stochastic = stochastic; #bool indicating whether stochasticity should be modeled
        self.age = 0; #seconds
        self.sensor_cro = 0;
        self.proteins = {"cro": 2, "cI": 100}
        self.history = {"age": [], "cro": [], "cI": []}
        self.repr_time = 60*60; #seconds for replication = 1 hr
        #Prepping for plotting
        self.plot = figure(title=plot_name, plot_height=400, plot_width=800);
        self.plot.title.text_font_size = "12pt"
        self.plot.title.align = "center"
        self.plot.xaxis.axis_label = "Minutes Elapsed";
        self.plot.xaxis.axis_label_text_font_size = "11pt"
        self.plot.xaxis.major_label_text_font_size = "10pt"
        self.plot.yaxis.axis_label = "Protein Concentration";
        self.plot.yaxis.axis_label_text_font_size = "11pt"
        self.plot.yaxis.major_label_text_font_size = "10pt"
        self.cro_series = self.plot.line([0], [0], color="green", legend="[cro]", line_width=3)
        self.cI_series = self.plot.line([0], [0], color="red", legend="[cI]", line_width=3)
        self.plot.legend.label_text_font_size = "11pt"
        self.render_plot()
        self.continuous_plotting = continuous_plotting; #bool :: Should the plot render every minute of sim?
    def render_plot(self):
        show(self.plot, notebook_handle=True, )
    def ff(self, nsteps, print_duration=1): #fastforward for nsteps
        start_time = time()
        [self.step() for s in range(nsteps)]
        if(print_duration):
            print("Computation time (nsteps="+str(nsteps)+"):", timedelta(seconds=time() - start_time))
        self.plot_history()
    def step(self):
        self.age+=1;
        self.history["age"].append(self.age/60)
        self.protein_behaviour()
        [self.history[prot_name].append(prot_conc) for prot_name, prot_conc in self.proteins.items()]
        if(self.age % self.repr_time == 0):
            self.divide()
        if(self.plot and self.age % 60 == 0 and self.continuous_plotting==1):
            self.plot_history()
    def plot_history(self):
        self.cro_series.data_source.data['x'] = self.history['age']
        self.cro_series.data_source.data['y'] = np.asarray(self.history['cro'], dtype=np.float32)
        self.cI_series.data_source.data['x'] = self.history['age']
        self.cI_series.data_source.data['y'] = np.asarray(self.history['cI'], dtype=np.float32)
        push_notebook()
    def protein_behaviour(self):
        dx = 1
        cro_form = subs_cro.subs([(cro, self.proteins["cro"]*2), (cI, self.proteins["cI"])]);
        new_cro = self.proteins["cro"] + dx*cro_form + self.sensor_cro*dx
        cI_form = subs_cI.subs([(cro, self.proteins["cro"]*1.8), (cI, self.proteins["cI"])]);
        new_cI = self.proteins["cI"] + dx*cI_form
        if(self.stochastic):
            new_cro += dx*(random()*2 - 1)*.1
            new_cI += dx*(random()*2 - 1)*.1
        self.proteins["cro"] = new_cro if new_cro > 0 else 0
        self.proteins["cI"] = new_cI if new_cI > 0 else 0
    def divide(self):
        self.proteins = {prot_name: prot_conc/2 for prot_name, prot_conc in self.proteins.items()}

In [ ]:
cell = Cell("Simulation Progress - Switching Failure", 0, 0)
cell.ff(60*10)
cell.sensor_cro = alpha_cro
cell.ff(60*10)
cell.sensor_cro = 0
cell.ff(60*30)
cell.ff(60*30)

In [ ]:
cell = Cell("Simulation Progress - Successful Switching", 0, 0)
cell.ff(60*20)
cell.proteins["cro"] += 15
cell.ff(60*80)
cell.proteins["cI"] += 15
cell.ff(60*60)
cell.plot_history()

In [ ]:
cell = Cell("Stochastic Simulation Progress - Incremental Switching Failure", 1, 0)
cell.ff(60*10)
def incr():
    cell.proteins["cro"]+=6;
    cell.ff(30, 0)
[incr() for i in range(0, 4)]
cell.ff(60*30)
cell.plot_history()

In [ ]:
cell = Cell("Stochastic Simulation Progress - Incremental Switching", 1, 0)
cell.ff(60*90)
def incr_cro():
    cell.proteins["cro"]+=6;
    cell.ff(30, 0)
[incr_cro() for i in range(0, 8)]
cell.ff(60*30)
cell.ff(60*30)
cell.ff(60*60)
def incr_cI():
    cell.proteins["cI"]+=6;
    cell.ff(30, 0)
[incr_cI() for i in range(0, 8)]
cell.ff(60*60)
cell.plot_history()

In [ ]:
cell = Cell("Stochastic Simulation Progress - Stability Testing", 1, 0)
cell.ff(60*60)
def incr_cro():
    cell.proteins["cro"]+=6;
    cell.ff(60, 0)
    cell.plot_history()
[incr_cro() for i in range(0, 4)]
cell.ff(60*120)
cell.plot_history()
cell.ff(60*120)
cell.plot_history()

In [ ]:
cell.ff(60*30)
cell.plot_history()

In [ ]:
cell.render_plot()
cell.plot_history()